In [ ]:
# specific to extracting information from word documents
import os
import zipfile
# other tools useful in extracting the information from our document
import re
# to pretty print our xml:
import xml.dom.minidom

from docx import *
import re
import json

import unicodedata

import spacy

In [ ]:
nlp = spacy.load("en_trf_distilbertbaseuncased_lg")

In [ ]:
from spacy_transformers import TransformersLanguage, TransformersWordPiecer, TransformersTok2Vec

In [ ]:
name = "distilbert-base-uncased"
nlp = TransformersLanguage(trf_name=name, meta={"lang": "en"})
nlp.add_pipe(nlp.create_pipe("sentencizer"))
nlp.add_pipe(TransformersWordPiecer.from_pretrained(nlp.vocab, name))
nlp.add_pipe(TransformersTok2Vec.from_pretrained(nlp.vocab, name))
print(nlp.pipe_names)  # ['sentencizer', 'trf_wordpiecer', 'trf_tok2vec']

In [ ]:
# courtesy of http://legalipsum.com/
filename = 'clause_tst_database2.csv'

In [ ]:
import csv

In [ ]:
# Check similarity - DONE
# Find strongest match - DONE
# Identify which does not have a strong match - DONE
# Build MS Word document, 
# first column with gen, and then 3 columns for each of the contracts, with remarks in 5th column - DONE
# Landscape mode - DONE

In [ ]:
clause_list = []
with open(filename) as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter=',')
    count = 0
    for row in csv_reader:
        count += 1
        print(count)
#         origin, cl_no, header, subheader, text, date = [ row[i] for i in row ]
        doc = nlp(row['text'])
#         print([ row[i] for i in row ] + [doc.vector])
#         print(doc.vector)
        clause_list.append([ row[i] for i in row ] + [doc.vector] )
        

In [ ]:
import pickle

In [ ]:
with open('clause_database_test2_processed.pkl', 'wb') as f:
    pickle.dump(clause_list, f)

In [ ]:
with open('clause_database_test2_processed.pkl', 'rb') as f:
    clause_list = pickle.load(f)

In [ ]:
clause_list[0]

In [ ]:
base = []
a = []
b = []
c = []
for clause in clause_list:
    if clause[0] == 'base':
        base.append(clause)
    elif clause[0] == 'a':
        a.append(clause)
    elif clause[0] == 'b':
        b.append(clause)
    elif clause[0] == 'c':
        c.append(clause)

In [ ]:
base_vectors = [ row[6] for row in base ]
a_vectors = [ row[6] for row in a ]
b_vectors = [ row[6] for row in b ]
c_vectors = [ row[6] for row in c ]

In [ ]:
from scipy.spatial import distance

In [ ]:
a[4][4]

In [ ]:
b[4][4]

In [ ]:
c[5][4]

In [ ]:
base[2][4]

In [ ]:
test = distance.cosine(base[2][6], a[4][6])

In [ ]:
1- test

In [ ]:
comparison_list = []
top_n = 5
for n, v1 in enumerate(base_vectors):
#     if n > 0:
        results_a = []
        for v2 in a_vectors:
            result = 1- distance.cosine(v1, v2)
            results_a.append(result)
        highest = max(results_a)
#         print(highest)
#         print(n)
#         print("THIS IS THE COMPARATOR*****" + gtcs[n][4] + "\n")
#         print("*****{}*****".format(cwb1mil[highest_index][4]))
#         h_index_a = [ j, i for i,j in enumerate(results_a) if j == highest ][0]
        top3_a = sorted( [(x,i) for (i,x) in enumerate(results_a)], reverse=True )[:top_n]
#         print(top3_a)
        h_index_a = top3_a[0]
#         print(h_index_a)
        
        results_b = []
        for v2 in b_vectors:
            result = 1- distance.cosine(v1, v2)
            results_b.append(result)
        highest = max(results_b)
#         h_index_b = [ j, i for i,j in enumerate(results_b) if j == highest ][0]
        top3_b = sorted( [(x,i) for (i,x) in enumerate(results_b)], reverse=True )[:top_n]
        h_index_b = top3_b[0]
    
        results_c = []
        for v2 in c_vectors:
            result = 1- distance.cosine(v1, v2)
            results_c.append(result)
        highest = max(results_c)
#         h_index_c = [ j, i for i,j in enumerate(results_c) if j == highest ][0]
        top3_c = sorted( [(x,i) for (i,x) in enumerate(results_c)], reverse=True )[:top_n]
        h_index_c = top3_c[0]
        
        remarks = ''
        
        threshold = 0.98
        
        if h_index_a[0] < threshold:
#             remarks += 'a below {}\n'.format(threshold)
            other_similar_clauses_a = [ cl[1] for cl in top3_a[1:] ]
            remarks += '\nContract A - Other similar clauses: ' + ', '.join([ a[i][1] for i in other_similar_clauses_a ]) + '\n'
        if h_index_b[0] < threshold:
#             remarks += 'b below {}\n'.format(threshold)
            other_similar_clauses_b = [ cl[1] for cl in top3_b[1:] ]
            remarks += '\nContract B - Other similar clauses: ' + ', '.join([ b[i][1] for i in other_similar_clauses_b ]) + '\n'
        if h_index_c[0] < threshold:
#             remarks += 'c below {}\n'.format(threshold)
            other_similar_clauses_c = [ cl[1] for cl in top3_c[1:] ]
            remarks += '\nContract C - Other similar clauses: ' + ', '.join([ c[i][1] for i in other_similar_clauses_c ]) + '\n'
        
        comparison_list.append((n, h_index_a, h_index_b, h_index_c, remarks, top3_a, top3_b, top3_c))

In [ ]:
from docx import Document
from docx.enum.section import WD_ORIENT
from docx.enum.section import WD_SECTION

In [ ]:
def change_orientation():
    current_section = document.sections[-1]
    new_width, new_height = current_section.page_height, current_section.page_width
#     new_section = document.add_section(WD_SECTION.NEW_PAGE)
    current_section.orientation = WD_ORIENT.LANDSCAPE
    current_section.page_width = new_width
    current_section.page_height = new_height

    return current_section

In [ ]:
document = Document()
change_orientation()
document.add_heading('DOCUMENT COMPARISON')
table = document.add_table(rows=1, cols=6)
table.style = 'LightShading-Accent1'

# populate header row --------
heading_cells = table.rows[0].cells
heading_cells[0].text = 'No'
heading_cells[1].text = 'Base Clause'
heading_cells[2].text = 'Contract A'
heading_cells[3].text = 'Contract B'
heading_cells[4].text = 'Contract C'
heading_cells[5].text = 'Remarks'

# add a data row for each item
for entry in comparison_list:
    
    cells = table.add_row().cells
    no, a_idx, b_idx, c_idx, remarks, _, _, _ = [ d for d in entry ]
    cells[0].text = str(no)
    cells[1].text = base[no][4]
    cells[2].text = a[cwb1mil_idx[1]][4]
    cells[3].text = b[cwa1mil_idx[1]][4]
    cells[4].text = c[engrsvcs_idx[1]][4]
    cells[5].text = str(remarks)

document.save('comparison_test.docx')